In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/chia_clinical_trials/

/content/drive/MyDrive/chia_clinical_trials


In [ ]:
!python -m pip install --user spacy==3.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 18.4 MB/s 
     |████████████████████████████████| 660 kB 65.6 MB/s 
     |████████████████████████████████| 10.1 MB 59.3 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.0 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.1.3 which is incompatible.


In [ ]:
import spacy
from spacy.tokens import DocBin
import random, math

In [ ]:
nlp = spacy.blank("en")

In [ ]:
doc_bin = DocBin()
doc_bin.from_disk("chia_rel_full.spacy")

In [ ]:
len(doc_bin)

1885

In [ ]:
docs = list(doc_bin.get_docs(nlp.vocab))

In [ ]:
docs[0].ents

(pathologically,
 invasive adenocarcinoma,
 ductal carcinoma in situ,
 segmental mastectomy,
 radiation therapy,
 local-regional disease)

## TRAIN TEST VALID SPLIT CODE

In [ ]:
# Random shuffling
random.shuffle(docs)
train_split = math.floor(len(docs) * 0.8)
train_docs = docs[:train_split]
test_docs = docs[train_split:]

dev_split = math.floor(len(test_docs) * 0.8)
dev_docs = test_docs[:dev_split]
test_docs = test_docs[dev_split:]

In [ ]:
print(len(train_docs))
print(len(dev_docs))
print(len(test_docs))
print(len(train_docs)+len(dev_docs)+len(test_docs))
print(len(docs))

1508
301
76
1885
1885


In [ ]:
# Using sklearn
from sklearn.model_selection import train_test_split

# you can add random state to preserve the split each time.
train_docs, test_docs = train_test_split(docs, test_size=0.2)
dev_docs, test_docs = train_test_split(test_docs, test_size=0.5)

In [ ]:
print(len(train_docs))
print(len(dev_docs))
print(len(test_docs))
print(len(train_docs)+len(dev_docs)+len(test_docs))
print(len(docs))

1508
188
189
1885
1885


## DUMP TO DISK

In [ ]:
train_doc_bin = DocBin()
for train_doc in train_docs:
    train_doc_bin.add(train_doc)
train_doc_bin.to_disk("./chia_rel_train.spacy")

In [ ]:
test_doc_bin = DocBin()
for test_doc in test_docs:
    test_doc_bin.add(test_doc)
test_doc_bin.to_disk("./chia_rel_test.spacy")

In [ ]:
dev_doc_bin = DocBin()
for dev_doc in dev_docs:
    dev_doc_bin.add(dev_doc)
dev_doc_bin.to_disk("./chia_rel_dev.spacy")

In [ ]:
pwd

'/content/drive/MyDrive/chia_clinical_trials'

In [ ]:
cd /content/drive/MyDrive/chia_clinical_trials/projects/tutorials/rel_component

/content/drive/MyDrive/chia_clinical_trials/projects/tutorials/rel_component


# RUN COMMAND

## a) Evaluating the REL model on CPU

In [ ]:
!python -m spacy project run all

ℹ Running workflow 'all'

==================================== data ====================================
ℹ Re-running 'data': spaCy minor version changed (3.4.1 in
project.lock, 3.1.3 current)
Running command: /usr/bin/python3 ./scripts/parse_data.py assets/annotations.jsonl data/chia_rel_train.spacy data/chia_rel_dev.spacy data/chia_rel_test.spacy
ℹ 102 training sentences from 43 articles, 209/2346 pos instances.
ℹ 27 dev sentences from 5 articles, 56/710 pos instances.
ℹ 20 test sentences from 6 articles, 30/340 pos instances.

================================= train_cpu =================================
Running command: /usr/bin/python3 -m spacy train configs/rel_tok2vec.cfg --output training --paths.train data/chia_rel_train.spacy --paths.dev data/chia_rel_dev.spacy -c ./scripts/custom_functions.py
ℹ Saving to output directory: training
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-08-30 06:55:55,759] [INFO] Set up nlp object from 

# Save REL Model

In [ ]:
output_dir = './rel-model'
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to ./rel-model


In [ ]:
# !python -m spacy download en_core_web_sm

In [ ]:
# !python -m spacy evaluate -o ./data/chia_rel_test.spacy en_core_web_sm ./data/chia_rel_train.spacy

## b) Evaluating the REL model on GPU

In [ ]:
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 1.1 MB 62.6 MB/s 
     |████████████████████████████████| 4.7 MB 54.4 MB/s 
     |████████████████████████████████| 6.3 MB 35.4 MB/s 
     |████████████████████████████████| 821 kB 49.9 MB/s 
     |████████████████████████████████| 6.6 MB 53.7 MB/s 
     |████████████████████████████████| 120 kB 62.5 MB/s 
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.1.3
    Uninstalling spacy-3.1.3:
      Successfully uninstalled spacy-3.1.3


In [ ]:
!python -m spacy project run all_gpu

ℹ Running workflow 'all_gpu'

==================================== data ====================================
ℹ Skipping 'data': nothing changed

================================= train_gpu =================================
Running command: /usr/bin/python3 -m spacy train configs/ner_rel_trf.cfg --output training --paths.train data/chia_rel_train.spacy --paths.dev data/chia_rel_dev.spacy -c ./scripts/custom_functions.py --gpu-id 0
ℹ Saving to output directory: training
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-30 12:42:43,812] [INFO] Set up nlp object from config
[2022-08-30 12:42:43,821] [INFO] Pipeline: ['transformer', 'ner', 'relation_extractor']
[2022-08-30 12:42:43,825] [INFO] Created vocabulary
[2022-08-30 12:42:43,826] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.w